# ETL Básico - Metadata-Sitios (Google Map)

En este archivo se enfoca en realizar la unificación de todos los archivos en el directorio `data/raw/google-maps/metadata-sitios/` y filtrar sólo los restaurantes en EEUU con el objetivo de guardar sólo la información relevante para hacer un EDA y un ETL profundos.

In [ ]:
import pandas as pd
import numpy as np
import warnings
import sys
import os

# Ignorar todas las advertencias
warnings.filterwarnings("ignore")

In [ ]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '../..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

# importar librerías personalizadas de ./src/utils/EDA.py
from src.utils.EDA import merge_files

Se utiliza la librería personalizada merge_files(path) para que lea todos los archivos en el directorio `data/raw/google_maps/metadata-sitios` y los retorne como un DataFrame unificado.

In [3]:
file_path = os.path.join(project_root, 'data/raw/google_maps/metadata-sitios')
df = merge_files(file_path)

Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/metadata-sitios\1.parquet
Archivo 1.parquet leído exitosamente con 275001 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/metadata-sitios\10.parquet
Archivo 10.parquet leído exitosamente con 275001 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/metadata-sitios\11.parquet
Archivo 11.parquet leído exitosamente con 275001 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/metadata-sitios\2.parquet
Archivo 2.parquet leído exitosamente con 275001 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/metadata-sitios\3.parquet
Archivo 3.parquet leído exitosamente con 275001 registros.
Leyendo archivo: d:\Henry-DataScience\LABS\Proyecto Grupal\Google-Yelp\data/raw/google_maps/metadata-sitios\4.p

In [4]:
# se visualizan las primeras tres filas del dataframe
df.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.29213,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [ ]:
#Verificar valores nulos
df.isna().sum()

name                     37
address               80511
gmap_id                   0
description         2770722
latitude                  0
longitude                 0
category              17419
avg_rating                0
num_of_reviews            0
price               2749808
hours                787405
MISC                 690834
state                746455
relative_results     295058
url                       0
dtype: int64

Se filtran los registros que contengan la palabra "restaurant" en la columna category. Los elementos nulos se obvian por ser una cantidad mínima considerando que los registros corresponden a todo el país.

In [ ]:
# Filtrado para obtener las filas que contengan la palabra "restaurant" en la columna category
filtered_df = df[df['category'].apply(lambda categories: categories is not None and any('restaurant' in item.lower() for item in categories))]

# retornar longitud del df y visualizar primeros elementos
print(len(filtered_df))
filtered_df.head(3)

212014


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
8,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,None,41.616079,-93.865487,"[Bakery, Health food restaurant]",4.7,21,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...","{'Accessibility': None, 'Activities': None, 'A...",Permanently closed,"[0x87ee974869295555:0x95f310d065882c9b, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...


En la columna state pueden apreciarse elementos con valor "Permanently closed" que indican que el negocio ya no abre sus puertas al público, por lo que pueden obviarse.

In [15]:
# Filtrado para obtener las filas que NO contengan el valor "Permanently closed" en la columna state
filtered_df = filtered_df[filtered_df['state'].apply(lambda state: state is None or (state.lower() != 'permanently closed'))]

# contar resultados y visualizar primeros elementos
print(len(filtered_df))
filtered_df.head(3)

161696


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,[Restaurant],4.0,8,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...


In [16]:
#Verificar valores nulos
filtered_df.isna().sum()

name                     0
address                897
gmap_id                  0
description         106983
latitude                 0
longitude                0
category                 0
avg_rating               0
num_of_reviews           0
price                82316
hours                22101
MISC                  1055
state                21263
relative_results     35729
url                      0
dtype: int64

## Se exportan sólo los restaurantes de EEUU

In [17]:
# se determina la ruta para exportar los datos de los restaurantes
export_path = os.path.join(project_root, 'data/raw/google_maps/restaurantes-EEUU.parquet')

# se exporta el df con restaurantes de EEUU a un archivo parquet
filtered_df.to_parquet(export_path, index=False, compression='gzip')